In [1]:
import torch 
import torch.nn as nn
from torch.nn import functional as F

import pandas as pd
import numpy as np

In [2]:
dataset = pd.read_csv('storepurchasedata_large.csv')

In [3]:
dataset.describe()

,Age,Salary,Purchased
count,1554.000000,1554.000000,1554.000000
mean,44.296010,57042.471042,0.694981
std,17.462458,21209.244800,0.460564
min,18.000000,20000.000000,0.000000
25%,27.000000,46000.000000,0.000000
50%,43.000000,60000.000000,1.000000
75%,62.000000,66000.000000,1.000000
max,69.000000,96000.000000,1.000000


In [4]:
dataset.head()

,Age,Salary,Purchased
0,18,20000,0
1,19,22000,0
2,20,24000,0
3,21,28000,0
4,22,60000,1


In [5]:
X = dataset.iloc[:, :-1].values
y = dataset.iloc[:,-1].values

In [6]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size =.20, random_state = 0)

In [7]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_test = sc.fit_transform(X_test)
X_train = sc.fit_transform(X_train)

Converting to datatype needed for Pytorch i.e **torch**

In [8]:
Xtrain_ = torch.from_numpy(X_train).float()
Xtest_ = torch.from_numpy(X_test).float()

In [9]:
ytrain_ = torch.from_numpy(y_train)
ytest_ = torch.from_numpy(y_test)

In [10]:
input_size = 2
output_size = 2
hidden_size = 10

In [11]:
class Net(nn.Module):
  def __init__(self):
    super(Net, self).__init__()
    #3 fully connected layer
    self.fc1 = torch.nn.Linear(input_size, hidden_size)
    self.fc2 = torch.nn.Linear(hidden_size, hidden_size)
    self.fc3 = torch.nn.Linear(hidden_size, output_size)

  def forward(self, X):
    X = torch.relu((self.fc1(X)))
    X = torch.relu((self.fc2(X)))
    X = self.fc3(X)

    return F.log_softmax(X,dim = 1)



In [12]:
model = Net()

In [13]:
import torch.optim as optim
optimizer = torch.optim.Adam(model.parameters(), lr = 0.01)
loss_fn = nn.NLLLoss()

In [14]:
epochs = 100

In [15]:
for epoch in range(epochs):
  optimizer.zero_grad()
  Ypred = model(Xtrain_)
  loss = loss_fn(Ypred, ytrain_)
  loss.backward()
  optimizer.step()
  print('Epoch', epoch, 'loss', loss.item())

Epoch 0 loss 0.6867042779922485
Epoch 1 loss 0.6480156779289246
Epoch 2 loss 0.616389811038971
Epoch 3 loss 0.5894403457641602
Epoch 4 loss 0.5661240220069885
Epoch 5 loss 0.5446780323982239
Epoch 6 loss 0.5234147906303406
Epoch 7 loss 0.5021082758903503
Epoch 8 loss 0.48070237040519714
Epoch 9 loss 0.45922768115997314
Epoch 10 loss 0.43820422887802124
Epoch 11 loss 0.4175567328929901
Epoch 12 loss 0.3971956968307495
Epoch 13 loss 0.3772589862346649
Epoch 14 loss 0.357785701751709
Epoch 15 loss 0.3388417065143585
Epoch 16 loss 0.3205574154853821
Epoch 17 loss 0.30301105976104736
Epoch 18 loss 0.28630584478378296
Epoch 19 loss 0.27047115564346313
Epoch 20 loss 0.25560423731803894
Epoch 21 loss 0.24176685512065887
Epoch 22 loss 0.22890450060367584
Epoch 23 loss 0.2170133888721466
Epoch 24 loss 0.2061106115579605
Epoch 25 loss 0.19599319994449615
Epoch 26 loss 0.18678580224514008
Epoch 27 loss 0.1783270686864853
Epoch 28 loss 0.17062607407569885
Epoch 29 loss 0.16373786330223083
Epoch 30 

In [16]:
list(model.parameters())

[Parameter containing:
 tensor([[-0.4224,  0.3579],
         [-0.0992, -0.8886],
         [ 0.1796,  1.2520],
         [ 0.3924,  0.3424],
         [-0.1295, -0.4419],
         [ 1.0906,  0.7044],
         [ 0.0664, -0.8690],
         [ 0.0292,  0.2105],
         [ 0.9715,  0.1018],
         [ 1.0927, -0.2861]], requires_grad=True), Parameter containing:
 tensor([-0.7494,  0.5765,  0.9352,  0.4908,  0.3080,  0.5467,  0.6685, -0.3529,
          0.3501,  0.0024], requires_grad=True), Parameter containing:
 tensor([[-0.2662,  0.4958, -0.1442, -0.4202,  0.3160,  0.0021,  0.3189, -0.3623,
          -0.1188, -0.9189],
         [-0.2555, -0.1110, -0.1580, -0.1602,  0.0535,  0.1910, -0.3473, -0.0023,
          -0.3521,  0.0645],
         [-0.0702,  0.2631, -0.2612, -0.0656,  0.3957, -0.7003,  0.3754, -0.3593,
          -0.3802, -0.8859],
         [-0.0034,  0.1825, -0.1902, -0.1264,  0.1560, -0.3316,  0.0691,  0.0792,
           0.0295, -0.3608],
         [-0.2876,  0.3681, -0.4080, -0.0332,  

In [17]:
torch.from_numpy(sc.transform(np.array([[40,20000]]))).float()

tensor([[-0.2650, -1.7622]])

In [18]:
y_cust_20_40000 = model(torch.from_numpy(sc.transform(np.array([[40,20000]]))).float())
y_cust_20_40000

tensor([[-2.2169e-03, -6.1128e+00]], grad_fn=<LogSoftmaxBackward>)

In [19]:
_, predicted_20_40000 = torch.max(y_cust_20_40000.data, -1)
predicted_20_40000

tensor([0])

In [20]:
torch.save(model, 'customer_buy.pt')

In [21]:
!ls

customer_buy.pt  sample_data  storepurchasedata_large.csv


In [22]:
restored_model = torch.load('customer_buy.pt')

In [23]:
torch.save(model.state_dict(), 'customer_buy_state_dict')

OrderedDict([('fc1.weight', tensor([[-0.4224,  0.3579],
                      [-0.0992, -0.8886],
                      [ 0.1796,  1.2520],
                      [ 0.3924,  0.3424],
                      [-0.1295, -0.4419],
                      [ 1.0906,  0.7044],
                      [ 0.0664, -0.8690],
                      [ 0.0292,  0.2105],
                      [ 0.9715,  0.1018],
                      [ 1.0927, -0.2861]])),
             ('fc1.bias',
              tensor([-0.7494,  0.5765,  0.9352,  0.4908,  0.3080,  0.5467,  0.6685, -0.3529,
                       0.3501,  0.0024])),
             ('fc2.weight',
              tensor([[-0.2662,  0.4958, -0.1442, -0.4202,  0.3160,  0.0021,  0.3189, -0.3623,
                       -0.1188, -0.9189],
                      [-0.2555, -0.1110, -0.1580, -0.1602,  0.0535,  0.1910, -0.3473, -0.0023,
                       -0.3521,  0.0645],
                      [-0.0702,  0.2631, -0.2612, -0.0656,  0.3957, -0.7003,  0.3754, -0.3593,
   